# Title: Nvidia Kaggle Compition

**Author : Jasmeen Sameer Shaikh**

**Github : [Jasmeen Guithub ID](http://github.com/jass2422)**


**LinkedIn:[Jasmeen LinkedIn ID](http://www.linkedin.com/in/jasmeen-shaikh-075b35240/)**



**This code implements a Multi-Layer Perceptron (MLP) using PyTorch to classify handwritten digits from the MNIST dataset. MNIST is a well-known benchmark in machine learning, consisting of 70,000 grayscale images of digits from 0 to 9. The goal is to build a deep neural network that learns meaningful patterns from the training data and generalizes well to unseen test data. The model consists of fully connected layers with ReLU activations and Dropout for regularization. It is trained using the Adam optimizer and CrossEntropy loss, which are well-suited for multiclass classification tasks. With proper architecture and training, this MLP achieves over 95% test accuracy on MNIST.**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Set device

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters

In [4]:
input_size = 28 * 28
hidden_sizes = [512, 256]
output_size = 10
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# MNIST Dataset

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 12.7MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 340kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.18MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.64MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



# Define the MLP Model

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        
        self.fc3 = nn.Linear(hidden_sizes[1], output_size)
    
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.dropout1(self.relu1(self.fc1(x)))
        x = self.dropout2(self.relu2(self.fc2(x)))
        x = self.fc3(x)
        return x

model = MLP().to(device)

# Loss and optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop

In [8]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)
        
        # Forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/10], Loss: 0.2377
Epoch [2/10], Loss: 0.1151
Epoch [3/10], Loss: 0.0838
Epoch [4/10], Loss: 0.0738
Epoch [5/10], Loss: 0.0612
Epoch [6/10], Loss: 0.0567
Epoch [7/10], Loss: 0.0525
Epoch [8/10], Loss: 0.0455
Epoch [9/10], Loss: 0.0450
Epoch [10/10], Loss: 0.0433


# Evaluation

In [9]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, targets in test_loader:
        data = data.to(device)
        targets = targets.to(device)
        
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 97.90%


# Conclusion

**In this project, we successfully developed a neural network using PyTorch that achieves high accuracy in digit classification on the MNIST dataset. Through the use of a well-structured MLP architecture, effective regularization (Dropout), and optimization techniques (Adam), the model reaches a test accuracy of over 97%. This performance demonstrates that even a relatively simple MLP, when properly tuned, can deliver robust results on image classification tasks. The approach can serve as a strong baseline for more complex models like CNNs or hybrid architectures in computer vision problems.**